In [1]:
import os
import json
from glob import glob
os.chdir("..")

In [2]:
path = "/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area individual files"
json_list = glob('/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/*.json')
json_list

['/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/zodiac_sign.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_weather_info.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/power.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_capital.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_trip_cost.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/count_letter.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_bmi.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_timezone.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area.json']

In [3]:
queries = []
answers = []
for file_path in json_list[0:]:
    with open(file_path, 'r') as file:
        text = json.load(file)
        text = text["queries"]
        for item in text:
            queries.append(item["query"])
            answers.append(json.dumps(item["answer"]))
        

In [4]:
import pandas as pd

# check schema 
df = pd.DataFrame({"queries": queries, "answers": answers})
df["functions"] = df["answers"].apply(lambda x: json.loads(x).get("function", None))

print(f"there are {len(df)} rows at the beginning")

there are 5867 rows at the beginning


In [5]:
#filtered non function
df = df[~df["functions"].isna()]
print(f"there are {len(df)} after filtering out non function")


there are 5866 after filtering out non function


In [6]:
# filter arguments
df["arguments"] = df["answers"].apply(lambda x: json.loads(x).get("arguments", None))
df = df[~df["arguments"].isna()]
df["arguments"] = df["arguments"].apply(json.dumps)

print(f"there are {len(df)} has arguments keys")


there are 5866 has arguments keys


In [7]:
from src.chi_functions import *
# from src.edward_functions import *
from src.MT_functions import * 
from src.kevin_function import * 


def call_function(info: pd.DataFrame):
    try:
        func_to_call = eval(info["functions"])
        
        arguments = json.loads(info["arguments"])
        return func_to_call(**arguments)
    except:
        return None
df["callable"] = df.apply(lambda x: call_function(x), axis=1)
df[df["callable"].isna()]["functions"].unique()



array(['power', 'get_capital', 'count_letter', 'get_timezone'],
      dtype=object)

In [14]:
from math import pow
df[df["callable"].isna()]
0**-3.131

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [39]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config

def generate_response(df, llm):
    query_output = df["queries"]
    function_output = df["callable"]
    messages = [
    ("system", "You are a bot and you should reply to the user based on the function calling return without letting the user know there is a function calling in the backend."),
    ("user", f"{query_output} and the function return is {function_output}")]
    response = llm.invoke(messages)
    return response.content

retry_config = Config(
    retries = {
        'max_attempts': 20,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries
llm = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-8b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)

df.iloc[:1000, :].apply(lambda x: generate_response(x, llm), axis=1)


ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.

array(['power', 'get_capital', 'count_letter', 'get_timezone'],
      dtype=object)

In [23]:
cou

<function math.isinf(x, /)>